# Modelo preditivo para a tendência do IBOVESPA

O objetivo desse projeto é desenvolver um modelo de Machine Learning capaz de prever se o índice IBOVESPA vai fechar em alta ou baixa no dia seguinte, com acuracidade mínima de 75% em um conjunto de teste composto pelos últimos 30 dias de dados.

> "O Ibovespa é o principal indicador de desempenho das ações negociadas na B3 e reúne as empresas mais importantes do mercado de capitais brasileiro. Foi criado em 1968 e, ao longo desses 50 anos, consolidou-se como referência para investidores ao redor do mundo." [Referência](https://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-amplos/ibovespa.htm)

## Importação das bibliotecas

In [ ]:
'''yfinance==0.2.65 pandas<2.0.0 numpy<2.0.0 scikit-learn==1.7.1 xgboost==3.0.2 matplotlib==3.10.3 ipython==9.4.0 jupyterlab==4.4.5 seaborn==0.13.2
pip install "numpy>=1.26,<2.0" "pandas_ta"'''

In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
#import pandas_ta as ta

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

## Aquisição dos dados

Utilizaremos os dados históricos do índice IBOVESPA, disponíveis publicamente no site do [br.investing](https://br.investing.com/indices/bovespa-historical-data)

In [2]:
input_path = '../data/raw/dados_historicos_ibovespa_2008-2025.csv'

df = pd.read_csv(input_path, thousands='.', decimal=',', parse_dates=['Data'], date_format='%d.%m.%Y', index_col='Data')
df = df.rename_axis('ds').sort_index()
df.tail()

,Último,Abertura,Máxima,Mínima,Vol.,Var%
ds,,,,,,
2025-06-12,137800,137127,137931,136175,"7,12B","0,49%"
2025-06-13,137213,137800,137800,136586,"8,63B","-0,43%"
2025-06-16,139256,137212,139988,137212,"7,62B","1,49%"
2025-06-17,138840,139256,139497,138293,"8,38B","-0,30%"
2025-06-18,138717,138844,139161,138443,"8,32B","-0,09%"


In [4]:
# informações gerais do dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4315 entries, 2008-01-18 to 2025-06-18
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Último    4315 non-null   int64 
 1   Abertura  4315 non-null   int64 
 2   Máxima    4315 non-null   int64 
 3   Mínima    4315 non-null   int64 
 4   Vol.      4314 non-null   object
 5   Var%      4315 non-null   object
dtypes: int64(4), object(2)
memory usage: 236.0+ KB


## Tratamento dos dados

Vamos transformar os dados brutos em um formato adequado para o treinamento de um modelo de Machine Learning.

In [5]:
# renomeando as colunas para os nomes padrões utilizados no mercado financeiro
colunas = {
  'Último': 'close',              # fechamento da negociação diária
  'Abertura': 'open',             # início da negociação diária
  'Máxima': 'high',               # valor máximo do dia
  'Mínima': 'low',                # valor mínimo do dia
  'Vol.': 'volume',               # volume de negociação diária
  'Var%': 'daily_return'          # variação percentual diária
}

df.rename(columns=colunas, inplace=True)

In [6]:
# Data mínima, máxima e total de anos do DF levando em conta os anos bissextos
print(f"Os dados vão de {df.index.min().date()} até {df.index.max().date()}, o que dá aproximadamente {(df.index.max() - df.index.min()).days / 365.25:.0f} anos")

Os dados vão de 2008-01-18 até 2025-06-18, o que dá aproximadamente 17 anos


In [7]:
# conferindo se há valores duplicados
df.duplicated().sum()

np.int64(0)

In [9]:
# conferindo se há valores nulos
df.isna().sum().sort_values(ascending=False)

volume          1
close           0
open            0
high            0
low             0
daily_return    0
dtype: int64

In [10]:
df[df['volume'].isnull()]

,close,open,high,low,volume,daily_return
ds,,,,,,
2016-02-10,40377,40592,40592,39960,NaN,"-0,53%"


In [11]:
def converter_volume(vol: str | float) -> float:
    """
    Converte uma string de volume com sufixos (K, M, B) para um número float.
    
    Parâmetro:
        vol (string | float): o valor a ser convertido (ex: '8,3M'). Pode ser uma string ou um np.nan (que é float).
        
    Retorna:
        float: o valor convertido ou np.nan caso não haja um valor.
    """
    if not isinstance(vol, str):
        return vol

    multiplicadores = {'K': 1e3, 'M': 1e6, 'B': 1e9}
    vol = vol.upper().replace(',', '.').strip()
    sufixo = vol[-1]

    if sufixo in multiplicadores:
        return float(vol[:-1]) * multiplicadores[sufixo]
    else:
        return float(vol)

df['volume'] = df['volume'].apply(converter_volume)

In [12]:
# substituir o volume nulo pela média do volume anterior e posterior daquela data
df['volume'] = df['volume'].interpolate()

In [13]:
# ajustando a coluna variação percentual diária, que contém o pct_change() do fechamento
df['daily_return'] = df['daily_return'].str.replace('%', '').str.replace(',', '.')
df['daily_return'] = round(df['daily_return'].astype(float) / 100, 4)
df.head()

,close,open,high,low,volume,daily_return
ds,,,,,,
2008-01-18,57506,57039,58291,56241,5810000.0,0.0082
2008-01-21,53709,57503,57503,53487,3570000.0,-0.0660
2008-01-22,56097,53705,56541,53610,3650000.0,0.0445
2008-01-23,54235,56098,56098,53011,3720000.0,-0.0332
2008-01-24,57463,54242,57675,54242,3800000.0,0.0595


In [14]:
# conferindo formato dos dados
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4315 entries, 2008-01-18 to 2025-06-18
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   close         4315 non-null   int64  
 1   open          4315 non-null   int64  
 2   high          4315 non-null   int64  
 3   low           4315 non-null   int64  
 4   volume        4315 non-null   float64
 5   daily_return  4315 non-null   float64
dtypes: float64(2), int64(4)
memory usage: 236.0 KB


## Engenharia de atributos

### Baseados em dados internos

Agora, vamos criar **variáveis preditoras** através de uma análise técnica do mercado financeiro, ao invés de uma análise fundamentalista. Em vez de dar ao modelo os dados brutos de abertura, máxima e mínima, vamos criar features para extrair a informação das relações entre eles.

Para as janelas de tempo, vamos utilizar as convenções do mercado financeiro para representar uma semana (5 dias), um mês (21 dias) e um trimestre (63 dias) de transações.

# Inicio Teste Jackson

In [72]:
# Cria um Data Frame limpo
df_prep = pd.DataFrame()

df_prep['Close'] = df['close'].copy()
delta = df_prep['Close'].diff() # Tira a diferença de um dia para o outro (d1 - d2)

threshold = 0.005 # desconsidera variações menos que 0.5%
df_prep['Target'] = np.where(
    delta > threshold, 1, np.where(delta < -threshold, 0, np.nan)
)

In [73]:
# Delta(diferença d1 - d2) e Return (variação diaria) adiantado um 1
df_prep['Delta'] = delta.shift(1)
df_prep['Return'] = df_prep['Close'].pct_change().shift(1)

In [74]:
# Função lags de Series
def make_lags(series: pd.Series, n_lags):
    return series.shift(n_lags)

# Função cria colunas defasadas
def make_n_lags(df, n_lags, column, step):
    for i in range(1, n_lags + 1, step):
        df[f"{column}_lag{i}"] = df[column].shift(i)
    return df

In [75]:
n_lags = 7

# Chama função cria colunas com lags
df_prep = make_n_lags(df_prep, n_lags, "Target", 1)
df_prep = make_n_lags(df_prep, n_lags, "Close", 2)
# Colunas criada algumas com defasagem de uma dia
df_prep['High'] = df['high'].shift(1).copy()
df_prep['Low'] = df['low'].shift(1).copy()
df_prep['Volatilidade'] = df_prep['High'] - df_prep['Low']
df_prep['Volatilidade_relativa'] = df_prep['High'] / df_prep['Low']
df_prep['Open'] = df['open'].copy()
# Média Movel Simples mais usadas
df_prep['MA9'] = df_prep['Close'].rolling(window=9).mean().shift(1)
df_prep['MA20'] = df_prep['Close'].rolling(window=20).mean().shift(1)
df_prep['MA50'] = df_prep['Close'].rolling(window=50).mean().shift(1)
df_prep['MA200'] = df_prep['Close'].rolling(window=20).mean().shift(1)
# Exclui tabela Close que não vai ser usada
df_prep.drop('Close', axis=1, inplace=True)
# Data Frame pronto para o modelo doprando linhas com valores na
df_model = df_prep.dropna().copy()
df_model

,Target,Delta,Return,Target_lag1,Target_lag2,Target_lag3,Target_lag4,Target_lag5,Target_lag6,Target_lag7,...,Close_lag7,High,Low,Volatilidade,Volatilidade_relativa,Open,MA9,MA20,MA50,MA200
ds,,,,,,,,,,,,,,,,,,,,,
2008-04-03,1.0,589.0,0.009383,1.0,1.0,1.0,0.0,0.0,1.0,1.0,...,61234.0,63816.0,62775.0,1041.0,1.016583,63367,61085.555556,61441.45,61241.54,61441.45
2008-04-04,1.0,811.0,0.012799,1.0,1.0,1.0,1.0,0.0,0.0,1.0,...,61415.0,64731.0,62793.0,1938.0,1.030863,64177,61662.000000,61418.75,61374.92,61418.75
2008-04-07,0.0,271.0,0.004223,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,60762.0,64630.0,63906.0,724.0,1.011329,64447,62176.777778,61492.30,61589.66,61492.30
2008-04-08,1.0,-270.0,-0.004190,0.0,1.0,1.0,1.0,1.0,1.0,0.0,...,60452.0,65409.0,63919.0,1490.0,1.023311,64180,62503.666667,61607.70,61751.24,61607.70
2008-04-09,0.0,364.0,0.005672,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,60968.0,64833.0,63453.0,1380.0,1.021748,64540,62850.888889,61834.75,61957.34,61834.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-12,1.0,692.0,0.005072,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,137546.0,137531.0,135628.0,1903.0,1.014031,137127,136662.555556,137815.35,134294.54,137815.35
2025-06-13,0.0,672.0,0.004901,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,137002.0,137931.0,136175.0,1756.0,1.012895,137800,136748.444444,137738.65,134445.34,137738.65
2025-06-16,1.0,-587.0,-0.004260,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,136236.0,137800.0,136586.0,1214.0,1.008888,137212,136795.777778,137639.95,134566.66,137639.95


In [76]:
# Libs Scikt-Learn
from sklearn.model_selection import train_test_split

# Separação das variáveis independentes e variáveis resposta
features = df_model.columns[1:]
target = 'Target'

X, y = df_model[features], df_model[target]
print(f'Linhas X {X.shape[0]}')
print(f'Linhas y {y.shape[0]}')

# Defini treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=42,
                                                    shuffle=False,
                                                    test_size=0.2)

Linhas X 4257
Linhas y 4257


In [77]:
print('Taxa variável resposta geral',y.mean())
print('Taxa variável resposta treino',y_train.mean())
print('Taxa variável resposta teste',y_test.mean())

Taxa variável resposta geral 0.5135071646699554
Taxa variável resposta treino 0.5145374449339207
Taxa variável resposta teste 0.5093896713615024


In [ ]:
# EDA (Exploration) - Só utilizaremos variáveis treino
X_train.isna().sum().sort_values(ascending=False)

Delta                    0
Return                   0
Target_lag1              0
Target_lag2              0
Target_lag3              0
Target_lag4              0
Target_lag5              0
Target_lag6              0
Target_lag7              0
Close_lag1               0
Close_lag3               0
Close_lag5               0
Close_lag7               0
High                     0
Low                      0
Volatilidade             0
Volatilidade_relativa    0
Open                     0
MA9                      0
MA20                     0
MA50                     0
MA200                    0
dtype: int64

In [ ]:
# Analise Bivariada
df_analise = X_train.copy()
df_analise[target] = y_train.copy()
sumario = df_analise.groupby(by=target).agg(['mean', 'median']).T
sumario

Target                                 0.0           1.0
Delta                 mean       72.626134    -44.265982
                      median     58.000000      6.500000
Return                mean        0.000916     -0.000267
                      median      0.000937      0.000107
Target_lag1           mean        0.528736      0.501142
                      median      1.000000      1.000000
Target_lag2           mean        0.523291      0.506279
                      median      1.000000      1.000000
Target_lag3           mean        0.526921      0.502854
                      median      1.000000      1.000000
Target_lag4           mean        0.517846      0.511416
                      median      1.000000      1.000000
Target_lag5           mean        0.499093      0.528539
                      median      0.000000      1.000000
Target_lag6           mean        0.519661      0.509703
                      median      1.000000      1.000000
Target_lag7           mean        0.516031      0.513699
                      median      1.000000      1.000000
Close_lag1            mean    69490.347852  69793.358447
                      median  62511.000000  63492.500000
Close_lag3            mean    69408.644888  69821.293950
                      median  62486.000000  63366.500000
Close_lag5            mean    69341.639443  69835.738584
                      median  62345.000000  63366.500000
Close_lag7            mean    69310.171204  69820.151826
                      median  62207.000000  63484.000000
High                  mean    70130.995160  70541.091895
                      median  63206.000000  64034.500000
Low                   mean    68748.528131  69097.919521
                      median  61814.000000  62810.500000
Volatilidade          mean     1382.467030   1443.172374
                      median   1165.000000   1209.000000
Volatilidade_relativa mean        1.021272      1.021892
                      median      1.018008      1.018574
Open                  mean    69489.104053  69796.039384
                      median  62503.000000  63488.500000
MA9                   mean    69368.264502  69812.430429
                      median  62383.333333  63493.055556
MA20                  mean    69314.566304  69732.935788
                      median  62379.600000  63299.925000
MA50                  mean    69148.258463  69527.152979
                      median  62948.120000  63217.740000
MA200                 mean    69314.566304  69732.935788
                      median  62379.600000  63299.925000

In [ ]:
from sklearn import tree
import matplotlib.pyplot as plt

arvore = tree.DecisionTreeClassifier(random_state=42)
arvore.fit(X_train, y_train)

,criterion,'gini'
,splitter,'best'
,max_depth,5
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,42
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


In [ ]:
'''plt.figure(dpi=400)
tree.plot_tree(arvore, feature_names=X_train.columns,
               class_names=[str(i) for i in arvore.classes_],
               filled=True)'''

In [88]:
# Associar cada feature e suas importâncias
features_importances = (pd.Series(arvore.feature_importances_,
                                  index=X_train.columns)
                                  .sort_values(ascending=False)
                                  .reset_index()
                                  )
features_importances['acum'] = features_importances[0].cumsum()
features_importances[features_importances['acum'] < 0.96]

,index,0,acum
0,Volatilidade_relativa,0.247958,0.247958
1,Return,0.144414,0.392372
2,Delta,0.131518,0.523890
3,Volatilidade,0.113939,0.637828
4,MA50,0.089230,0.727058
5,MA200,0.080530,0.807588
6,Target_lag4,0.033594,0.841182
7,Close_lag3,0.033071,0.874253
8,Close_lag5,0.032909,0.907162
9,MA9,0.032151,0.939313


In [90]:
# Separa dados para uma respostamais confiavel e não embara os dados
from sklearn.model_selection import TimeSeriesSplit
# Variável para separação de dados sem embaralhar
tscv = TimeSeriesSplit(n_splits=5)

# importe do Pipeline de dados e dos modelos de teste
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler


In [91]:
pipe_lr = Pipeline(
    [
        ('scaler', StandardScaler()),
        ('clf', LogisticRegression(solver='liblinear'))
    ]
)

# Separando melhores valor dos modelos
param_grid_lr = {
    'clf__C': [0.001, 0.01, 0.1, 1, 10, 100], # Regularização
    'clf__penalty': ['l1', 'l2'], # Tipo de penalização
    'clf__solver': ['liblinear'] # necessário para suportar l1
}

grid_lr = GridSearchCV(pipe_lr, param_grid_lr, cv=tscv)  # Varre os hiperparametros para escolher os melhores
grid_lr.fit(X_train, y_train) # Treina o melhor modelo

,estimator,Pipeline(step...liblinear'))])
,param_grid,"{'clf__C': [0.001, 0.01, ...], 'clf__penalty': ['l1', 'l2'], 'clf__solver': ['liblinear']}"
,scoring,None
,n_jobs,None
,refit,True
,cv,TimeSeriesSpl...est_size=None)
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,copy,True


In [ ]:
pipe_xgb = Pipeline(
    [
        ('scaler', StandardScaler()),
        ('clf', XGBClassifier()),
    ]
)

param_grid_xgb = {
    'n_estimators': 500,               # Menos árvores, mais rápido
    'max_depth': 3,                    # Shallow trees para evitar overfitting
    'learning_rate': 0.05,             # Learning rate moderado
    'subsample': 0.9,                  # Mais dados por árvore
    'colsample_bytree': 0.9,
    'colsample_bylevel': 0.9,
    'reg_alpha': 0.1,                  # Regularização menor
    'reg_lambda': 0.5,
    'min_child_weight': 1,             # Permite splits menores
    'gamma': 0,                        # Sem poda adicional
    'scale_pos_weight': 1,
    'random_state': 42,
    'tree_method': 'hist',
}

#grid_xgb = GridSearchCV(pipe_xgb, param_grid_xgb, cv=tscv)
grid_xgb = RandomizedSearchCV(pipe_xgb, param_grid_xgb, cv=tscv) # Ele tira mostras aleatórias dos dados ai ele não passa por todos os estados
grid_xgb.fit(X_train, y_train)

,estimator,"Pipeline(step...=None, ...))])"
,param_distributions,"{'clf__colsample_bytree': [0.6, 0.8, ...], 'clf__gamma': [0, 0.1, ...], 'clf__learning_rate': [0.01, 0.05, ...], 'clf__max_depth': [3, 5, ...], ...}"
,n_iter,10
,scoring,None
,n_jobs,None
,refit,True
,cv,TimeSeriesSpl...est_size=None)
,verbose,0
,pre_dispatch,'2*n_jobs'
,random_state,None
,error_score,nan


In [93]:
print("Logistic Regression:")
print("Melhores parâmetros:", grid_lr.best_params_) # Salva o melhor modelo nesse best_params_
print("Acurácia:", accuracy_score(y_test, grid_lr.predict(X_test)))
print(classification_report(y_test, grid_lr.predict(X_test)))

Logistic Regression:
Melhores parâmetros: {'clf__C': 100, 'clf__penalty': 'l1', 'clf__solver': 'liblinear'}
Acurácia: 0.4800469483568075
              precision    recall  f1-score   support

         0.0       0.47      0.43      0.45       418
         1.0       0.49      0.53      0.51       434

    accuracy                           0.48       852
   macro avg       0.48      0.48      0.48       852
weighted avg       0.48      0.48      0.48       852



In [94]:
print("\nXGBoost:")
print("Melhores parâmetros:", grid_xgb.best_params_)
print("Acurácia:", accuracy_score(y_test, grid_xgb.predict(X_test)))
print(classification_report(y_test, grid_xgb.predict(X_test)))


XGBoost:
Melhores parâmetros: {'clf__subsample': 0.8, 'clf__reg_lambda': 1, 'clf__reg_alpha': 1, 'clf__n_estimators': 100, 'clf__max_depth': 5, 'clf__learning_rate': 0.3, 'clf__gamma': 0, 'clf__colsample_bytree': 0.6}
Acurácia: 0.49765258215962443
              precision    recall  f1-score   support

         0.0       0.49      0.54      0.51       418
         1.0       0.51      0.45      0.48       434

    accuracy                           0.50       852
   macro avg       0.50      0.50      0.50       852
weighted avg       0.50      0.50      0.50       852



In [95]:
baseline_pred = delta.shift(1)

baseline_pred = np.where(
    baseline_pred > threshold, 1, np.where(baseline_pred < - threshold, 0, np.nan)
)[-30:]

In [ ]:
print('\nBaseline:')
print('Acurácia:', accuracy_score(y_test, baseline_pred))
print(classification_report(y_test, baseline_pred))

In [97]:
# acurácia dos modelos

# regressão logística
log_reg_accuracy = accuracy_score(y_test, grid_lr.predict(X_test))

# xgboost
xgb_accuracy = accuracy_score(y_test, grid_xgb.predict(X_test))

baseline_accuracy = accuracy_score(y_test, baseline_pred)

# criando a tabela de comparação
tabela_comparativa = pd.DataFrame(columns=['Modelo', 'Acurácia'])

tabela_comparativa.loc[0] = ['Baseline', f"{baseline_accuracy*100:.2f}%"]
tabela_comparativa.loc[1] = ['Logistic Regression', f"{log_reg_accuracy*100:.2f}%"]
tabela_comparativa.loc[2] = ['XGBoost', f"{xgb_accuracy*100:.2f}%"]

tabela_comparativa

ValueError: Found input variables with inconsistent numbers of samples: [852, 30]

# Fim Teste Jackson

In [14]:
# indicadores de momentum

# retorno percentual de 1 a 5 dias (tendência de curto prazo)
for lag in range(1, 6):
    df[f'return_lag_{lag}'] = df['daily_return'].shift(lag)

# retorno percentual acumulado de uma semana, um mês e um trimestre
for period in [5, 21, 63]:
    df[f'momentum_{period}'] = df['close'].pct_change(period)

A média móvel é uma técnica que suaviza as flutuações de curto prazo e destaca tendências de longo prazo. Enquanto a média móvel simples calcula a média aritmética em um período predeterminado, a média móvel exponencial dá mais peso aos valores mais recentes, tornando-a mais sensível a novas informações.

Quando uma média móvel de curto prazo cruza acima de uma média móvel de longo prazo, pode sugerir uma mudança de tendência de decrescimento para crescimento.

In [15]:
# 2. Seleção de Features (X)
# Adicionar lag do 'close' como feature

df['close_lag1'] = df['close'].shift(1)
df['close_lag3'] = df['close'].shift(3)
df['close_lag5'] = df['close'].shift(5)
df['close_lag7'] = df['close'].shift(7)

In [16]:
# indicadores de tendência

# média móvel simples de um mês (tendência de médio prazo)
df['sma_21'] = ta.sma(df['close'], length=21)

# média móvel exponencial de um trimestre (tendência de longo prazo)
df['ema_50'] = ta.ema(df['close'], length=50)

Para a média móvel exponencial, é uma convenção utilizar 50 dias ou 200 dias para tendências de longo prazo. [Referência](https://www.investopedia.com/terms/e/ema.asp#citation-4)

In [17]:
# indicadores de osciladores

# índice de força relativa (IFR) varia de 0 a 100
df['rsi_14'] = ta.rsi(df['close'], length=14)

Por padrão, utiliza-se um período de 14 dias para o IFR. [Referência](https://blog.quantinsti.com/rsi-indicator/)

In [18]:
# indicadores de volatilidade

# average true range
df['atr_14'] = ta.atr(df['high'], df['low'], df['close'], length=14)

Por padrão, utiliza-se um período de 14 dias para o ATR. [Referência](https://www.investopedia.com/terms/a/atr.asp)

Por padrão, utiliza-se um período de 20 dias (um mês) para e 2 desvios-padrão para as bandas para conter cerca de 95% dos dados. [Referência](https://www.infomoney.com.br/guias/bandas-bollinger/)

In [19]:
# indicadores de volume

# on-balance volume 
df['obv'] = ta.obv(df['close'], df['volume'])

Decidimos não usar as features de volume, uma vez que são dados com muito ruído e que vão de uma escala de mil até bilhão. (vide notebook EDA)

In [20]:
# indicadores de data

# dia da semana (segunda=0 a sexta=4)
df['day_of_week'] = df.index.dayofweek

# dia do mês (1 a 30 ou 31)
df['day_of_month'] = df.index.day

# mês (janeiro=1 a dezembro=12)
df['month'] = df.index.month

# ano (pode não ser bom para previsões de curto prazo)
df['year'] = df.index.year

### Baseados em dados externos

> "O desempenho do mercado de ações ao redor do mundo afeta diretamente o comportamento da bolsa brasileira" [Referência](https://investnews.com.br/guias/bolsas-internacionais-quais-sao-como-afetam-ibovespa/) e [Referência](https://einvestidor.estadao.com.br/mercado/como-bolsas-internacionais-afetam-b3-ibovespa/)

* S&P 500 (`^GSPC`): Um dos principais índices da NYSE, o S&P 500 inclui 500 empresas líderes e representa aproximadamente 80% da capitalização de mercado disponível.
* Índice SSE Composite (`000001.SS`): O principal índice do Shanghai Stock Exchange, o SSEC, abarca ações de grandes empresas da China, como a Alibaba (BABA34), o Bank of China e a China Petrol.
* Euronext 100 (`^N100`): O índice Euronext 100, engloba companhia de alguns países da União Europeia, e tem ações de empresas como L’Oréal, Louis Vuitton e Renault.

* Petróleo tipo Brent (`BZ=F`): O valor do barril da commodity interfere no preço das ações. No caso da Petrobras, o barril do tipo Brent é a principal referência. [Referência](https://borainvestir.b3.com.br/noticias/empresas/quais-os-principais-fatores-que-afetam-as-acoes-da-petrobras/)

In [ ]:
'''tickers = ['^GSPC', '000001.SS', '^N100', 'BZ=F']
df_global = yf.download(tickers, start='2010-01-01', end='2025-06-19', auto_adjust=True, multi_level_index=False)
df_global.head()'''

NameError: name 'yf' is not defined

In [ ]:
'''# selecionar apenas a coluna de fechamento
df_global = df_global['Close']

# lidar com dados ausentes (feriados de outros mercados)
df_global.ffill(inplace=True)'''

In [ ]:
'''# calcular retorno diário com defasagem (do dia anterior)
external_features = pd.DataFrame(index=df_global.index)

external_features ['eua_return_lag1'] = df_global['^GSPC'].pct_change().shift(1)
external_features ['china_return_lag1'] = df_global['000001.SS'].pct_change().shift(1)
external_features ['europe_return_lag1'] = df_global['^N100'].pct_change().shift(1)
external_features ['oil_return_lag1'] = df_global['BZ=F'].pct_change().shift(1)

external_features.head()'''

,eua_return_lag1,china_return_lag1,europe_return_lag1,oil_return_lag1
Date,,,,
2010-01-04,NaN,NaN,NaN,NaN
2010-01-05,NaN,NaN,NaN,NaN
2010-01-06,0.003116,0.011844,0.000832,0.005866
2010-01-07,0.000546,-0.008520,0.001004,0.016131
2010-01-08,0.004001,-0.018880,-0.000730,-0.004640


In [ ]:
'''# juntar com o dataframe do ibovespa
df_merged = df.merge(external_features, left_index=True, right_index=True, how='left')'''

## Exportação dos dados

In [22]:
# selecionando apenas 10 anos de dados
#df_10years = df_merged.loc[df.index >= '2015-06-17']
df_10years = df[df.index >= '2015-06-17']
df_10years.head()

,close,open,high,low,volume,daily_return,return_lag_1,return_lag_2,return_lag_3,return_lag_4,...,close_lag7,sma_21,ema_50,rsi_14,atr_14,obv,day_of_week,day_of_month,month,year
ds,,,,,,,,,,,,,,,,,,,,,
2015-06-17,53249,53698,53755,52965,3090000.0,-0.0084,0.0106,-0.0039,-0.0064,-0.0035,...,52810.0,53785.285714,53919.575856,44.356672,889.081392,78500090.0,2,17,6,2015
2015-06-18,54239,53251,54352,53214,2750000.0,0.0186,-0.0084,0.0106,-0.0039,-0.0064,...,52816.0,53725.285714,53932.102293,51.525407,906.861293,81250090.0,3,18,6,2015
2015-06-19,53749,54236,54236,53479,2950000.0,-0.0090,0.0186,-0.0084,0.0106,-0.0039,...,53876.0,53670.428571,53924.921811,48.214460,896.371201,78300090.0,4,19,6,2015
2015-06-22,53864,53750,54342,53655,2430000.0,0.0021,-0.0090,0.0186,-0.0084,0.0106,...,53689.0,53611.000000,53922.532720,49.042076,881.416115,80730090.0,0,22,6,2015
2015-06-23,53772,53865,54361,53772,2710000.0,-0.0017,0.0021,-0.0090,0.0186,-0.0084,...,53348.0,53582.190476,53916.629476,48.375998,860.529250,78020090.0,1,23,6,2015


In [23]:
df_10years.isnull().sum()

close           0
open            0
high            0
low             0
volume          0
daily_return    0
return_lag_1    0
return_lag_2    0
return_lag_3    0
return_lag_4    0
return_lag_5    0
momentum_5      0
momentum_21     0
momentum_63     0
close_lag1      0
close_lag3      0
close_lag5      0
close_lag7      0
sma_21          0
ema_50          0
rsi_14          0
atr_14          0
obv             0
day_of_week     0
day_of_month    0
month           0
year            0
dtype: int64

In [24]:
# Salvar o CSV já limpo e processado com a data no índice
output_path = '../data/processed/dados_historicos_ibovespa_2015-2025_processed.csv'

df_10years.to_csv(output_path, index=True)